# **0. Azure Open AI Endpoint and Key:  GPT - 4o - Mini Model**

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = "#hidden"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a vibrant city filled with iconic landmarks, cultural experiences, and picturesque neighborhoods. Here’s a list of must-see attractions and activities during your visit:

### Iconic Landmarks
1. **Eiffel Tower**: Visit this iconic symbol of Paris, and don’t forget to go up for stunning views of the city.
2. **Louvre Museum**: Home to thousands of artworks, including the Mona Lisa and the Venus de Milo.
3. **Notre-Dame Cathedral**: Although under restoration, the facade is still impressive; you can also explore the Île de la Cité.
4. **Sacré-Cœur Basilica**: Located on Montmartre Hill, enjoy the beautiful architecture and panoramic views. 
5. **Arc de Triomphe**: Climb to the top for a great view of the Champs-Élysées.

### Museums and Galleries
6. **Musée d'Orsay**: Famous for its collection of Impressionist and Post-Impressionist masterpieces.
7. **Centre Pompidou**: A modern art museum with a striking architectural design.
8. **Musée de l'Orangerie**: Known for Monet's Water

In [4]:
pip install ir_datasets tqdm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.0 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=a094d29e64712eb3131aa5c158a23bc05281cf8f424e5361e012616ad917ffe5
  Stored in directory: /root/.cache/pip/wheels/f6/85/c2/9f0f621def52a1d5db7d29984f81e45f9fb6dfeb1a4eb6e31c
  Created wheel for cbor: filename=cbor-1.0.0-cp312-cp312-linux_x86_64.whl size=55019 sha256=3cf082d14422820030b7e4f7ca77f1f72b16594b64ab19b5bfe6135223d4e916
  Stored in directory: /root/.cache/pip/wheels/44/3e/21/a739cbcc331a1ab45c326d6edbdac6118de4402f6076e30ff1
Successfully built warc3-wet-clueweb09 cbor


# **1. Collecting Base Query Documents from msmarco and beir**

In [ ]:

import ir_datasets, random, json
from tqdm import tqdm


SAMPLES = {
    "msmarco-passage/train": 400,            # open-domain search
    "beir/cqadupstack/programmers": 100,     # StackOverflow-style Q&A
    "beir/fever": 100,                       # fact verification
    "beir/scidocs": 50,                      # academic domain
    "beir/quora": 50,                        # question paraphrasing
}

random.seed(42)

def sample_dataset(name, sample_size):
    print(f"\n📘 Processing {name}")
    ds = ir_datasets.load(name)

    for needed in ["queries_iter", "docs_iter", "qrels_iter"]:
        if not hasattr(ds, needed):
            raise AttributeError(f"Dataset {name} missing: {needed}")

    queries = {q.query_id: q.text for q in ds.queries_iter()}
    docs = {d.doc_id: d.text for d in ds.docs_iter()}
    qrels = list(ds.qrels_iter())

    # mapping query → list of relevant doc IDs
    pos_map = {}
    for q in qrels:
        if q.relevance > 0:
            pos_map.setdefault(q.query_id, []).append(q.doc_id)

    valid_qids = list(pos_map.keys())
    if not valid_qids:
        print(f"⚠️ No valid qrels for {name}")
        return []

    chosen_qids = random.sample(valid_qids, min(sample_size, len(valid_qids)))
    results = []

    for qid in tqdm(chosen_qids):
        query = queries.get(qid, "")
        pos_docs = pos_map.get(qid, [])
        for did in pos_docs[:3]:
            if did in docs:
                results.append({
                    "dataset": name,
                    "query_id": qid,
                    "query": query,
                    "document": docs[did],
                    "relevance": 1
                })

    print(f"✅ Collected {len(results)} pairs from {name}")
    return results


all_data = []
for name, n in SAMPLES.items():
    try:
        all_data.extend(sample_dataset(name, n))
    except Exception as e:
        print(f"⚠️ Skipping {name}: {e}")


with open("1_base_dataset_questq.jsonl", "w", encoding="utf8") as f:
    for item in all_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"\n Done! Collected {len(all_data)} total query–doc pairs.")
print("Output saved to base_dataset_questq.jsonl")


#**2. Adding Attributes to Base Dataset**

In [ ]:
import json, time, random
from tqdm import tqdm
from openai import AzureOpenAI

endpoint = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
api_version = "2024-12-01-preview"
deployment = "gpt-4o-mini"
api_key = "#hidden"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=api_key
)


with open("1_base_dataset_questq.jsonl") as f:
    base_data = [json.loads(l) for l in f]

FEWSHOT = """
Examples:
1️⃣ Query: "what is the purpose of DNA replication"
Document: "DNA replication ensures each cell gets an exact copy of the DNA during cell division."
Attributes:
{
 "audience": "Student",
 "keyword": ["Biology", "Genetics"],
 "format": "Academic Paper",
 "language": "English",
 "length": "Short",
 "source": "Wikipedia"
}

2️⃣ Query: "price of a bushel of wheat"
Document: "Interactive chart of historical daily wheat prices..."
Attributes:
{
 "audience": "Researcher",
 "keyword": ["Economics", "Agriculture"],
 "format": "Report",
 "language": "English",
 "length": "Short",
 "source": "NewsSite"
}
"""

# Attribute generation
def get_attributes(query, document, retries=2):
    prompt = f"""
You are labeling information retrieval data using InfoSearch-style attributes.

{FEWSHOT}

Now label this new pair.
Return ONLY a valid JSON dictionary (no explanations, no markdown).

Query: {query}
Document: {document[:800]}
JSON:
"""
    for _ in range(retries):
        try:
            resp = client.chat.completions.create(
                model=deployment,
                temperature=0.4,
                max_tokens=250,
                messages=[{"role": "user", "content": prompt}]
            )
            content = resp.choices[0].message.content.strip()

            start = content.find("{")
            end = content.rfind("}") + 1
            json_part = content[start:end]
            attrs = json.loads(json_part)

            required = {"audience", "keyword", "format", "language", "length", "source"}
            if required.issubset(attrs.keys()):
                return attrs
        except Exception as e:
            time.sleep(1)
            continue
    return None


enriched = []
for i, item in enumerate(tqdm(base_data, desc="Annotating")):
    attrs = get_attributes(item["query"], item["document"])
    if attrs:
        item.update(attrs)
        enriched.append(item)

with open("2_base_with_multi_attri.jsonl", "w", encoding="utf8") as f:
    for e in enriched:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

print(f" Created multi_attr_dataset.jsonl with {len(enriched)} labeled pairs.")


# **3. Multi-Attribute Instructed + Reversed Query Generation + Document Rewriting + Hard Negative Generation**


In [ ]:
!pip install openai aiohttp nest_asyncio -q

import json, re, time, asyncio, nest_asyncio, random, copy
from openai import AsyncAzureOpenAI
nest_asyncio.apply()


API_KEY = "#hidden"
ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
API_VERSION = "2024-12-01-preview"
MODEL = "gpt-4o-mini"

client = AsyncAzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=ENDPOINT,
    api_version=API_VERSION
)


# Rate Control for Token Usage
MAX_CONCURRENT_REQUESTS = 6
REQUESTS_PER_MINUTE = 1000
SLEEP_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE
SAVE_INTERVAL = 50


def safe_json_parse(text):
    try:
        return json.loads(text)
    except Exception:
        instructed = re.search(r'"?instructed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        reversed_q = re.search(r'"?reversed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        pos = re.search(r'"?positive[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        neg = re.search(r'"?hard[_ ]?negative[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        return {
            "instructed_query": instructed.group(1).strip() if instructed else "",
            "reversed_query": reversed_q.group(1).strip() if reversed_q else "",
            "positive_doc": pos.group(1).strip() if pos else "",
            "hard_negative_doc": neg.group(1).strip() if neg else ""
        }

def build_instruct_prompt(query, attributes):
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are generating search queries with multiple document-level attributes.

Given a base query and its attributes, produce:
1. An instructed version that naturally includes all given attributes.
2. A reversed instructed version that logically negates them.

Return a JSON object:
{{
  "instructed_query": "...",
  "reversed_query": "..."
}}

Base query: "{query}"
Attributes: {attr_text}
"""

def build_doc_prompt(document, attributes):
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are refining a document for an information-retrieval benchmark.

Task 1 – Rewrite the base document so that it **fully satisfies all given attributes**.
Task 2 – Create one **hard negative document** that is **topically similar** but violates one or two attributes.

Return only JSON:
{{
  "positive_doc": "... rewritten version ...",
  "hard_negative_doc": "... violating version ..."
}}

Base document:
\"\"\"{document}\"\"\"
Attributes: {attr_text}
"""

def sample_attr_combinations(entry, num_combinations=5):
    all_attrs = {
        "audience": entry.get("audience", ""),
        "format": entry.get("format", ""),
        "language": entry.get("language", ""),
        "length": entry.get("length", ""),
        "source": entry.get("source", "")
    }
    non_empty = {k: v for k, v in all_attrs.items() if v}
    num_combinations = random.randint(3, 5)

    combos = []
    for _ in range(num_combinations):
        chosen = random.sample(list(non_empty.keys()), k=min(random.randint(2, 3), len(non_empty)))
        combos.append({k: non_empty[k] for k in chosen})
    return combos

#Async Generation
async def generate_instructed_queries():
    input_file = "2_base_with_multi_attri.jsonl"
    output_file = "3_instructed_reverse_queries_with_attri.jsonl"

    with open(input_file, "r") as f:
        base_data = [json.loads(line) for line in f]

    print(f"📘 Loaded {len(base_data)} core queries")
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def process_combination(base_entry, combo, combo_id):
        new_entry = copy.deepcopy(base_entry)
        new_entry["attributes"] = combo
        new_entry["combo_id"] = combo_id

        prompt = build_instruct_prompt(base_entry["query"], combo)
        try:
            response = await client.chat.completions.create(
                model=MODEL,
                temperature=0.4,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            parsed = safe_json_parse(response.choices[0].message.content)
            new_entry["instructed_query"] = parsed.get("instructed_query", "")
            new_entry["reversed_query"] = parsed.get("reversed_query", "")
            new_entry["query_type"] = "expanded"
            return new_entry
        except Exception as e:
            print(f" Error on {base_entry.get('query_id')}: {e}")
            return None

    tasks = []
    for entry in base_data:
        combos = sample_attr_combinations(entry)
        for i, combo in enumerate(combos):
            tasks.append(process_combination(entry, combo, i+1))

    async def limited_task(task):
        async with semaphore:
            res = await task
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return res

    wrapped_tasks = [limited_task(t) for t in tasks]
    for i, coro in enumerate(asyncio.as_completed(wrapped_tasks), 1):
        res = await coro
        if res:
            processed.append(res)
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f" Auto-saved {i} combinations...")

    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f" Generated {len(processed)} instructed + reversed combinations!")

# Document Rewriting
async def rewrite_documents():
    input_file = "3_instructed_reverse_queries_with_attri.jsonl"
    output_file = "4_rewritten_docs_hard_negatives_with_attri.jsonl"

    with open(input_file, "r") as f:
        data = [json.loads(line) for line in f]

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def process_doc(entry):
        attributes = entry.get("attributes", {})
        prompt = build_doc_prompt(entry.get("document", ""), attributes)
        try:
            response = await client.chat.completions.create(
                model=MODEL,
                temperature=0.5,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            parsed = safe_json_parse(response.choices[0].message.content)
            violated = random.sample(
                list(attributes.keys()), k=min(random.randint(1, 2), len(attributes))
            ) if attributes else []
            entry["positive_doc"] = parsed.get("positive_doc", "")
            entry["hard_negative_doc"] = parsed.get("hard_negative_doc", "")
            entry["violated_attributes"] = violated
            return entry
        except Exception as e:
            print(f" Error on query_id={entry.get('query_id')}: {e}")
            return None

    tasks = [process_doc(e) for e in data]
    async def limited(entry_task):
        async with semaphore:
            res = await entry_task
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return res

    wrapped_tasks = [limited(t) for t in tasks]
    for i, coro in enumerate(asyncio.as_completed(wrapped_tasks), 1):
        res = await coro
        if res:
            processed.append(res)
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f"💾 Saved {i} rewritten docs...")

    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f" Finished rewriting {len(processed)} documents!")

await generate_instructed_queries()
await rewrite_documents()


📘 Loaded 715 core queries
💾 Auto-saved 50 combinations...
💾 Auto-saved 100 combinations...
💾 Auto-saved 150 combinations...
💾 Auto-saved 200 combinations...
💾 Auto-saved 250 combinations...
💾 Auto-saved 300 combinations...
💾 Auto-saved 350 combinations...
💾 Auto-saved 400 combinations...
💾 Auto-saved 450 combinations...
💾 Auto-saved 500 combinations...
💾 Auto-saved 550 combinations...
💾 Auto-saved 600 combinations...
💾 Auto-saved 650 combinations...
💾 Auto-saved 700 combinations...
💾 Auto-saved 750 combinations...
💾 Auto-saved 800 combinations...
💾 Auto-saved 850 combinations...
💾 Auto-saved 900 combinations...
💾 Auto-saved 950 combinations...
💾 Auto-saved 1000 combinations...
💾 Auto-saved 1050 combinations...
💾 Auto-saved 1100 combinations...
💾 Auto-saved 1150 combinations...
💾 Auto-saved 1200 combinations...
💾 Auto-saved 1250 combinations...
💾 Auto-saved 1300 combinations...
💾 Auto-saved 1350 combinations...
💾 Auto-saved 1400 combinations...
💾 Auto-saved 1450 combinations...
💾 Auto-s

In [ ]:
import json

input_file = "4_rewritten_docs_hard_negatives_with_attri.jsonl"
output_file = "final_sorted.jsonl"

with open(input_file, "r") as f:
    data = [json.loads(line) for line in f if line.strip()]

def sort_key(entry):
    qid = str(entry.get("query_id", "")).strip()

    return (0, int(qid)) if qid.isdigit() else (1, qid.lower())

data.sort(key=sort_key)

with open(output_file, "w") as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f" Sorted {len(data)} records by query_id.")
print(f" Output saved to: {output_file}")


 # **4. Evaluation Process**

In [ ]:
!pip install pyserini tqdm numpy pandas -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.8/178.8 MB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 15.0 MB/s eta 0:0

#**4.1 Sparse Retrieval Models**

#BM25

In [5]:
!pip install rank_bm25 tqdm numpy pandas -q


In [ ]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from rank_bm25 import BM25Okapi
import json, numpy as np, pandas as pd, os
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

# CONFIG
DATA_QDOC = "query-doc.json"
DATA_QUERIES = "final_sorted.jsonl"
RESULTS_DIR = "results_generic"
os.makedirs(RESULTS_DIR, exist_ok=True)


# METRIC FUNCTIONS

def compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev):
    if (Rins_rank < Rori_rank and Sins > Sori) and (Rrev_rank > Rori_rank and Srev < Sori):
        return 1
    return 0

def compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, sat, viol, K=10, N=1):
    try:
        if Rins_rank <= Rori_rank < Rrev_rank:
            if Rori_rank <= N and Rins_rank == 1:
                f = 1.0
            elif Rori_rank <= K:
                f = (1 - np.sqrt((Rori_rank - Rins_rank) / K)) * (1 / np.sqrt(max(Rins_rank, 1)))
            else:
                f = 0.01
            return f * (sat / m)
        else:
            if Rrev_rank < Rori_rank < Rins_rank:
                p = -1.0
            elif Rori_rank <= Rins_rank:
                p = (Rori_rank - Rins_rank) / max(Rins_rank, 1)
            elif Rrev_rank <= Rori_rank:
                p = (Rrev_rank - Rori_rank) / max(Rori_rank, 1)
            else:
                p = -0.5
            return p * (viol / m)
    except:
        return 0.0

def compute_MDCR_bm25(query, top_docs, attributes, top_k=10):
    if not attributes:
        return 0.0, 0

    attr_keywords = [f"{name} {value}".lower().split() for name, value in attributes.items()]

    soft_scores, strict_scores = [], []

    for doc_text in top_docs[:top_k]:
        doc_tokens = set(word_tokenize(doc_text.lower()))
        sims = []
        for attr_tokens in attr_keywords:
            overlap = len(set(attr_tokens) & doc_tokens) / len(attr_tokens)
            sims.append(overlap)

        mean_sim = np.mean(sims)
        threshold = max(0.4, mean_sim)

        mdcr_soft = mean_sim
        mdcr_strict = int(all(s >= threshold for s in sims))

        soft_scores.append(mdcr_soft)
        strict_scores.append(mdcr_strict)

    return float(max(soft_scores)), int(max(strict_scores))

# BM25 EVALUATION FUNCTION
def evaluate_bm25(top_k=10):
    print("\n Evaluating BM25 baseline...")

    # Load Data
    with open(DATA_QDOC, "r") as f:
        qdoc = json.load(f)
    with open(DATA_QUERIES, "r") as f:
        queries = [json.loads(line) for line in f]

    # Build corpus
    corpus, doc_ids, qid_to_docs = [], [], {}
    for entry in qdoc:
        qid = entry["query_id"]
        qid_to_docs[qid] = {}
        for doc in entry["documents"]:
            text = doc["text"].strip().lower()
            corpus.append(text)
            doc_id = f"{qid}_{doc['doc_id']}"
            doc_ids.append(doc_id)
            qid_to_docs[qid][text] = doc_id

    print(f"Building BM25 index for {len(corpus)} documents ...")
    tokenized_corpus = [word_tokenize(doc) for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    def retrieve_topk(query):
        scores = bm25.get_scores(word_tokenize(query.lower()))
        top_idx = np.argsort(scores)[::-1][:top_k]
        return [(doc_ids[i], corpus[i], float(scores[i])) for i in top_idx]

    def find_rank(docid, ranking):
        for i, (d, _, _) in enumerate(ranking):
            if d == docid:
                return i + 1
        return len(ranking) + 1

    results = []
    for q in tqdm(queries, desc="Evaluating BM25"):
        try:
            qid = q["query_id"]
            pos_text = q.get("positive_doc", "").strip().lower()
            attrs = q.get("attributes", {})
            m = len(attrs) or 1

            pos_doc = qid_to_docs.get(qid, {}).get(pos_text)
            if not pos_doc:
                pos_doc = list(qid_to_docs.get(qid, {}).values())[0]

            Rori = retrieve_topk(q["query"])
            Rins = retrieve_topk(q["instructed_query"])
            Rrev = retrieve_topk(q["reversed_query"])

            Rori_rank = find_rank(pos_doc, Rori)
            Rins_rank = find_rank(pos_doc, Rins)
            Rrev_rank = find_rank(pos_doc, Rrev)

            Sori = Rori[Rori_rank - 1][2] if Rori_rank <= len(Rori) else 0
            Sins = Rins[Rins_rank - 1][2] if Rins_rank <= len(Rins) else 0
            Srev = Rrev[Rrev_rank - 1][2] if Rrev_rank <= len(Rrev) else 0

            # Compute metrics
            mSICR_val = compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev)
            mWISE_val = compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, m, 0)
            top_docs_instructed = [text for _, text, _ in Rins]
            mdcr_soft, mdcr_strict = compute_MDCR_bm25(q["instructed_query"], top_docs_instructed, attrs)

            results.append({
                "query_id": qid,
                "mSICR": mSICR_val,
                "mWISE": mWISE_val,
                "MDCR_soft": mdcr_soft,
                "MDCR_strict": mdcr_strict
            })

        except Exception as e:
            print(f"⚠ Skipped query {q.get('query_id')} due to: {e}")
            continue

    # Aggregate results
    df = pd.DataFrame(results)
    metrics = {
        "model": "BM25",
        "mSICR": df["mSICR"].mean(),
        "mWISE": df["mWISE"].mean(),
        "MDCR_soft": df["MDCR_soft"].mean(),
        "MDCR_strict": df["MDCR_strict"].mean(),
        "queries_evaluated": len(df)
    }

    save_path = os.path.join(RESULTS_DIR, "BM25_metrics.csv")
    df.to_csv(save_path, index=False)

    print(f"\n✅ Saved per-query results → {save_path}")
    print(f"✅ Aggregate metrics for BM25:")
    print(json.dumps(metrics, indent=2))
    return metrics

# RUN
evaluate_bm25()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

Evaluating BM25 baseline... 
Building BM25 index for 9576 documents ... 
Evaluating BM25:  49%|████▉     | 4746/9596 [19:48<11:06,  7.27it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating BM25:  50%|████▉     | 4751/9596 [19:48<07:36, 10.62it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating BM25:  50%|████▉     | 4757/9596 [19:49<07:25, 10.87it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating BM25:  50%|████▉     | 4761/9596 [19:49<07:30, 10.73it/s]⚠ Skipped query 447321 due to: 'dict' object has no 

#**4.2 Dense Retrieval Models**

#Bge-Large-v1.5

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm
import os

# CONFIG
device = "cuda" if torch.cuda.is_available() else "cpu"
DATA_QDOC = "query-doc.json"
DATA_QUERIES = "final_sorted.jsonl"
RESULTS_DIR = "results_generic"
os.makedirs(RESULTS_DIR, exist_ok=True)


# Metric Functions

def compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev):
    if (Rins_rank < Rori_rank and Sins > Sori) and (Rrev_rank > Rori_rank and Srev < Sori):
        return 1
    return 0


def compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated, K=10, N=1):
    try:
        # Reward case
        if Rins_rank <= Rori_rank < Rrev_rank:
            if Rori_rank <= N and Rins_rank == 1:
                freward = 1.0
            elif Rori_rank <= K:
                freward = (1 - np.sqrt((Rori_rank - Rins_rank) / K)) * (1 / np.sqrt(max(Rins_rank, 1)))
            else:
                freward = 0.01
            return freward * (conditions_satisfied / m)

        # Penalty case
        else:
            if Rrev_rank < Rori_rank < Rins_rank:
                fpenalty = -1.0
            elif Rori_rank <= Rins_rank:
                fpenalty = (Rori_rank - Rins_rank) / max(Rins_rank, 1)
            elif Rrev_rank <= Rori_rank:
                fpenalty = (Rrev_rank - Rori_rank) / max(Rori_rank, 1)
            else:
                fpenalty = -0.5
            return fpenalty * (conditions_violated / m)

    except Exception:
        return 0.0


def compute_MDCR(model, top_docs, attributes, top_k=10):
    if not attributes:
        return 0.0, 0

    attr_embs = []
    for attr_name, attr_value in attributes.items():
        desc = f"The document should reflect {attr_name} = {attr_value}."
        emb = model.encode(desc, convert_to_tensor=True)
        attr_embs.append(emb)

    soft_scores, strict_scores = [], []

    # For each top-K document, evaluate attribute satisfaction
    for doc_text in top_docs:
        doc_emb = model.encode(doc_text, convert_to_tensor=True)
        sims = [float(util.cos_sim(doc_emb, a_emb)) for a_emb in attr_embs]
        sims = np.clip(sims, 0, 1)


        mean_sim = np.mean(sims)
        std_sim = np.std(sims)
        threshold = np.clip(mean_sim + 0.1 * std_sim, 0.5, 0.9)

        # Soft score
        if np.max(sims) - np.min(sims) < 1e-6:
            mdcr_soft = float(mean_sim)
        else:
            mdcr_soft = float((mean_sim - np.min(sims)) / (np.max(sims) - np.min(sims) + 1e-6))

        # Strict score
        mdcr_strict = int(all(s >= threshold for s in sims))

        soft_scores.append(mdcr_soft)
        strict_scores.append(mdcr_strict)

    return float(max(soft_scores)), int(max(strict_scores))

# Evaluation Function
def evaluate_embedding_model(model_name, batch_size=32, top_k=10):
    print(f"\n Loading Embedding Model: {model_name}")
    model = SentenceTransformer(model_name, device=device)

    # Load Data
    with open(DATA_QDOC, "r") as f:
        qdoc = json.load(f)
    with open(DATA_QUERIES, "r") as f:
        queries = [json.loads(line) for line in f]

    # Build Corpus
    corpus, doc_ids, qid_to_docs = [], [], {}
    for entry in qdoc:
        qid = entry["query_id"]
        qid_to_docs[qid] = {}
        for doc in entry["documents"]:
            text = doc["text"].strip().lower()
            corpus.append(text)
            doc_id = f"{qid}_{doc['doc_id']}"
            doc_ids.append(doc_id)
            qid_to_docs[qid][text] = doc_id

    print(f"Encoding {len(corpus)} documents ...")
    corpus_embs = model.encode(corpus, convert_to_tensor=True, batch_size=batch_size, show_progress_bar=True)

    def retrieve_topk(query):
        query_emb = model.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_emb, corpus_embs)[0]
        top_results = torch.topk(cos_scores, top_k)
        return [(doc_ids[idx], corpus[idx], float(cos_scores[idx])) for idx in top_results.indices]

    def find_rank(docid, ranking):
        for i, (d, _, _) in enumerate(ranking):
            if d == docid:
                return i + 1
        return len(ranking) + 1

    # Evaluation Loop
    results = []

    for q in tqdm(queries, desc=f"Evaluating {model_name}"):
        try:
            qid = q["query_id"]
            pos_text = q.get("positive_doc", "").strip().lower()
            attrs = q.get("attributes", {})
            m = len(attrs) or 1

            # Identify positive document
            pos_doc = qid_to_docs.get(qid, {}).get(pos_text)
            if not pos_doc:
                pos_emb = model.encode(pos_text, convert_to_tensor=True)
                cos_scores = util.cos_sim(pos_emb, corpus_embs)[0]
                pos_doc = doc_ids[int(torch.argmax(cos_scores))]

            # Retrieve top-k results
            Rori = retrieve_topk(q["query"])
            Rins = retrieve_topk(q["instructed_query"])
            Rrev = retrieve_topk(q["reversed_query"])

            Rori_rank = find_rank(pos_doc, Rori)
            Rins_rank = find_rank(pos_doc, Rins)
            Rrev_rank = find_rank(pos_doc, Rrev)

            Sori = Rori[Rori_rank - 1][2] if Rori_rank <= len(Rori) else 0
            Sins = Rins[Rins_rank - 1][2] if Rins_rank <= len(Rins) else 0
            Srev = Rrev[Rrev_rank - 1][2] if Rrev_rank <= len(Rrev) else 0

            # Metrics
            conditions_satisfied = m
            conditions_violated = 0

            mSICR_val = compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev)
            mWISE_val = compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated)

            # Adaptive MDCR
            top_docs_instructed = [text for _, text, _ in Rins]
            mdcr_soft, mdcr_strict = compute_MDCR(model, top_docs_instructed, attrs, top_k=top_k)

            results.append({
                "query_id": qid,
                "mSICR": mSICR_val,
                "mWISE": mWISE_val,
                "MDCR_soft": mdcr_soft,
                "MDCR_strict": mdcr_strict
            })

        except Exception as e:
            print(f"⚠ Skipped query {q.get('query_id')} due to: {e}")
            continue

    # Aggregate results
    df = pd.DataFrame(results)
    metrics = {
        "model": model_name,
        "mSICR": df["mSICR"].mean(),
        "mWISE": df["mWISE"].mean(),
        "MDCR_soft": df["MDCR_soft"].mean(),
        "MDCR_strict": df["MDCR_strict"].mean(),
        "queries_evaluated": len(df)
    }

    save_path = os.path.join(RESULTS_DIR, f"{model_name.replace('/', '_')}_metrics.csv")
    df.to_csv(save_path, index=False)

    print(f"\n✅ Saved per-query results → {save_path}")
    print(f"✅ Aggregate metrics for {model_name}:")
    print(json.dumps(metrics, indent=2))
    return metrics


In [ ]:
evaluate_embedding_model("BAAI/bge-large-en-v1.5")


Loading Embedding Model: BAAI/bge-large-en-v1.5
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
modules.json: 100% 349/349 [00:00<00:00, 32.0kB/s]config_sentence_transformers.json: 100% 124/124 [00:00<00:00, 16.0kB/s]README.md:  94.6k/? [00:00<00:00, 6.09MB/s]sentence_bert_config.json: 100% 52.0/52.0 [00:00<00:00, 5.01kB/s]config.json: 100% 779/779 [00:00<00:00, 73.9kB/s]model.safetensors: 100% 1.34G/1.34G [00:12<00:00, 132MB/s]tokenizer_config.json: 100% 366/366 [00:00<00:00, 36.0kB/s]vocab.txt:  232k/? [00:00<00:00, 12.7M

#E5-Large-v2

In [ ]:
model_name = "intfloat/e5-large-v2"
evaluate_embedding_model(model_name)


Loading Embedding Model: intfloat/e5-large-v2 
Encoding 9576 documents ...
Batches: 100% 300/300 [03:55<00:00,  3.57it/s]Evaluating intfloat/e5-large-v2:  27%|██▋       | 2614/9596 [19:58<53:36,  2.17it/s]⚠ Skipped query 192005 due to: 'dict' object has no attribute 'strip'
Evaluating intfloat/e5-large-v2:  49%|████▉     | 4748/9596 [36:37<20:18,  3.98it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating intfloat/e5-large-v2:  50%|████▉     | 4758/9596 [36:39<16:22,  4.92it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating intfloat/e5-large-v2: 100%|██████████| 9596/9596 [1:09:44<00:00,  2.29it/s]
✅ Saved per-query results → results_generic/intfloat_e5-large-v2_metrics.csv
✅Aggregate metrics for intfloat/e5-large-v2: 
{
  "model": "intfloat/e5-large-v2",
  "mSICR": 0.0154488517745302

#Instructor-XL

In [ ]:
evaluate_embedding_model("hkunlp/instructor-large")

Loading Embedding Model: hkunlp/instructor-large
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret HF_TOKEN does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
modules.json: 100%
 461/461 [00:00<00:00, 49.8kB/s]
config_sentence_transformers.json: 100%
 122/122 [00:00<00:00, 8.99kB/s]
README.md:
 66.3k/? [00:00<00:00, 6.75MB/s]
sentence_bert_config.json: 100%
 53.0/53.0 [00:00<00:00, 5.49kB/s]
config.json:
 1.53k/? [00:00<00:00, 160kB/s]
pytorch_model.bin: 100%
 1.34G/1.34G [00:13<00:00, 79.8MB/s]
tokenizer_config.json:
 2.41k/? [00:00<00:00, 241kB/s]
model.safetensors: 100%
 1.34G/1.34G [0

#GTE-Qwen2

In [ ]:
evaluate_embedding_model("thenlper/gte-small")


Loading Embedding Model: thenlper/gte-small 
modules.json: 100%
 385/385 [00:00<00:00, 46.6kB/s]
README.md:
 68.1k/? [00:00<00:00, 5.96MB/s]
sentence_bert_config.json: 100%
 57.0/57.0 [00:00<00:00, 6.67kB/s]
config.json: 100%
 583/583 [00:00<00:00, 68.9kB/s]
model.safetensors: 100%
 66.7M/66.7M [00:00<00:00, 93.1MB/s]
tokenizer_config.json: 100%
 394/394 [00:00<00:00, 40.0kB/s]
vocab.txt:
 232k/? [00:00<00:00, 11.2MB/s]
tokenizer.json:
 712k/? [00:00<00:00, 37.9MB/s]
special_tokens_map.json: 100%
 125/125 [00:00<00:00, 16.0kB/s]
config.json: 100%
 190/190 [00:00<00:00, 20.8kB/s]
Encoding 9576 documents ...
Batches: 100%
 300/300 [00:23<00:00, 30.16it/s]
Evaluating thenlper/gte-small:  49%|████▉     | 4746/9596 [13:42<10:02,  8.06it/s]⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
⚠ Skipped query 447321 due to: 'dict' object has no attribute 'strip'
Evaluating thenlper/gte-small:  50%|████▉     | 4751/9596 [13:43<06:38, 12.17it/s]⚠ Skipped query 447321 due to: 'd

# GritLM

In [ ]:
evaluate_embedding_model("Alibaba-NLP/gte-base-en-v1.5")

Loading Embedding Model: Alibaba-NLP/gte-base-en-v1.5
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret HF_TOKEN does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
modules.json: 100%
 229/229 [00:00<00:00, 25.3kB/s]
README.md: 
 72.3k/? [00:00<00:00, 6.28MB/s]
sentence_bert_config.json: 100%
 54.0/54.0 [00:00<00:00, 6.11kB/s]
config.json: 
 1.35k/? [00:00<00:00, 145kB/s]
configuration.py: 
 7.13k/? [00:00<00:00, 705kB/s]
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain 

# E5-Mistral-ins

In [3]:
!pip install huggingface_hub tqdm pandas numpy --quiet

import os
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from huggingface_hub import InferenceClient

# CONFIG
HF_ENDPOINT = "#hidden"
HF_TOKEN = "#hidden"

MODEL_NAME = "intfloat/e5-mistral-7b-instruct"
QUERY_DOC_FILE = "query-doc.json"
FINAL_SORTED_FILE = "final_sorted.jsonl"

EMBED_DIR = Path("embeddings_remote")
RESULTS_DIR = Path("results_remote")
EMBED_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

BATCH_SIZE = 2
TOP_K = 100
DEVICE = "remote_endpoint"

print(f"✅ Using remote inference endpoint: {HF_ENDPOINT}")
print(f"Running evaluation for model: {MODEL_NAME}")


def load_data(query_doc_file, final_sorted_file):
    with open(query_doc_file, "r", encoding="utf-8") as f:
        qdoc = json.load(f)
    queries = []
    with open(final_sorted_file, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return qdoc, queries


def build_corpus_from_qdoc(qdoc):
    corpus, doc_ids, qid_to_docs = [], [], {}
    for entry in qdoc:
        qid = entry["query_id"]
        qid_to_docs[qid] = {}
        for doc in entry["documents"]:
            text = doc["text"].strip()
            key = text.lower()
            doc_id = f"{qid}_{doc['doc_id']}"
            corpus.append(text)
            doc_ids.append(doc_id)
            qid_to_docs[qid][key] = doc_id
    return corpus, doc_ids, qid_to_docs


def remote_embed(sentences, client, batch_size=2, retries=3, sleep_time=10):
    embeddings = []
    for i in tqdm(range(0, len(sentences), batch_size), desc="Embedding via Endpoint"):
        batch = [s.strip() for s in sentences[i:i + batch_size] if s and s.strip()]
        if not batch:
            continue

        success = False
        for attempt in range(retries):
            try:
                resp = client.feature_extraction(batch)

                if isinstance(resp, np.ndarray):
                    arr = resp.astype(np.float32)
                elif isinstance(resp, list):
                    if isinstance(resp[0], list) and isinstance(resp[0][0], list):
                        resp = resp[0]
                    arr = np.array(resp, dtype=np.float32)
                else:
                    raise ValueError(f"Unexpected response type: {type(resp)}")

                if arr.ndim != 2:
                    raise ValueError(f"Unexpected embedding shape: {arr.shape}")

                embeddings.append(arr)
                success = True
                break

            except Exception as e:
                print(f"⚠️ Failed batch {i}, attempt {attempt+1}/{retries}: {e}")
                time.sleep(sleep_time * (2 ** attempt))

        if not success:
            print(f"❌ Skipping batch {i} after {retries} retries.")

    if not embeddings:
        raise RuntimeError("No successful embeddings retrieved.")
    return np.vstack(embeddings)


def encode_corpus_cached(model_name, corpus, client):
    safe_name = model_name.replace("/", "_")
    out_dir = EMBED_DIR / safe_name
    out_dir.mkdir(exist_ok=True)
    emb_file = out_dir / "corpus_emb.npy"
    if emb_file.exists():
        print(f"✅ Loading cached corpus embeddings for {model_name}")
        return np.load(emb_file)
    print(f" Encoding corpus for {model_name} via endpoint ...")
    embs = remote_embed(corpus, client, batch_size=BATCH_SIZE)
    np.save(emb_file, embs)
    return embs


def cosine_sim(a, b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return float(np.dot(a, b))


def find_rank(target_docid, ranked_list):
    try:
        return ranked_list.index(target_docid) + 1
    except ValueError:
        return len(ranked_list) + 1


# Metric Functions
def compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev):
    if (Rins_rank < Rori_rank and Sins > Sori) and (Rrev_rank > Rori_rank and Srev < Sori):
        return 1
    return 0


def compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated, K=10, N=1):
    try:
        if Rins_rank <= Rori_rank < Rrev_rank:
            if Rori_rank <= N and Rins_rank == 1:
                freward = 1.0
            elif Rori_rank <= K:
                freward = (1 - np.sqrt((Rori_rank - Rins_rank) / K)) * (1 / np.sqrt(max(Rins_rank, 1)))
            else:
                freward = 0.01
            return freward * (conditions_satisfied / m)
        else:
            if Rrev_rank < Rori_rank < Rins_rank:
                fpenalty = -1.0
            elif Rori_rank <= Rins_rank:
                fpenalty = (Rori_rank - Rins_rank) / max(Rins_rank, 1)
            elif Rrev_rank <= Rori_rank:
                fpenalty = (Rrev_rank - Rori_rank) / max(Rori_rank, 1)
            else:
                fpenalty = -0.5
            return fpenalty * (conditions_violated / m)
    except Exception:
        return 0.0


def compute_MDCR_remote(client, top_docs, attributes):
    if not attributes:
        return 0.0, 0

    attr_embs = []
    for k, v in attributes.items():
        desc = f"The document should reflect {k} = {v}."
        attr_emb = remote_embed([desc], client, batch_size=1)[0]
        attr_embs.append(attr_emb)

    soft_scores, strict_scores = [], []
    for doc_text in top_docs:
        doc_emb = remote_embed([doc_text], client, batch_size=1)[0]
        sims = [cosine_sim(doc_emb, a_emb) for a_emb in attr_embs]
        sims = np.clip(sims, 0, 1)

        mean_sim = np.mean(sims)
        std_sim = np.std(sims)
        threshold = np.clip(mean_sim + 0.1 * std_sim, 0.5, 0.9)

        mdcr_soft = float((mean_sim - np.min(sims)) / (np.max(sims) - np.min(sims) + 1e-6))
        mdcr_strict = int(all(s >= threshold for s in sims))

        soft_scores.append(mdcr_soft)
        strict_scores.append(mdcr_strict)

    return float(max(soft_scores)), int(max(strict_scores))


# Evaluation
def evaluate_model_remote(model_name, corpus, doc_ids, qid_to_docs, queries):
    client = InferenceClient(base_url=HF_ENDPOINT, token=HF_TOKEN)
    results = []

    corpus_embs = encode_corpus_cached(model_name, corpus, client)
    corpus_embs = corpus_embs / np.linalg.norm(corpus_embs, axis=1, keepdims=True).clip(min=1e-9)

    def retrieve_topk(text):
        q_emb = remote_embed([text], client, batch_size=1)[0]
        q_emb = q_emb / np.linalg.norm(q_emb)
        sims = corpus_embs @ q_emb
        idxs = np.argsort(-sims)[:TOP_K]
        return [(doc_ids[i], corpus[i], float(sims[i])) for i in idxs]

    for q in tqdm(queries, desc=f"Evaluating {model_name}"):
        try:
            qid = q["query_id"]
            pos_text = q.get("positive_doc", "")
            if isinstance(pos_text, dict):
                pos_text = pos_text.get("text", "")
            pos_text = str(pos_text).strip().lower()
            if not pos_text:
                continue

            pos_doc = qid_to_docs.get(qid, {}).get(pos_text)
            if not pos_doc:
                continue

            attrs = q.get("attributes", {})
            m = len(attrs) or 1
            conditions_satisfied, conditions_violated = m, 0

            Rori = retrieve_topk(q["query"])
            Rins = retrieve_topk(q["instructed_query"])
            Rrev = retrieve_topk(q["reversed_query"])

            Rori_rank = find_rank(pos_doc, [d for d, _, _ in Rori])
            Rins_rank = find_rank(pos_doc, [d for d, _, _ in Rins])
            Rrev_rank = find_rank(pos_doc, [d for d, _, _ in Rrev])

            Sori = Rori[Rori_rank - 1][2] if Rori_rank <= len(Rori) else 0
            Sins = Rins[Rins_rank - 1][2] if Rins_rank <= len(Rins) else 0
            Srev = Rrev[Rrev_rank - 1][2] if Rrev_rank <= len(Rrev) else 0

            mSICR_val = compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev)
            mWISE_val = compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated)
            mdcr_soft, mdcr_strict = compute_MDCR_remote(client, [t for _, t, _ in Rins], attrs)

            results.append({
                "query_id": qid,
                "mSICR": mSICR_val,
                "mWISE": mWISE_val,
                "MDCR_soft": mdcr_soft,
                "MDCR_strict": mdcr_strict
            })

        except Exception as e:
            print(f"⚠️ Skipped query {q.get('query_id')} due to: {e}")
            continue

    if not results:
        print(f"⚠️ No results for {model_name}")
        return None

    df = pd.DataFrame(results)
    metrics = {
        "model": model_name,
        "mSICR": df["mSICR"].mean(),
        "mWISE": df["mWISE"].mean(),
        "MDCR_soft": df["MDCR_soft"].mean(),
        "MDCR_strict": df["MDCR_strict"].mean(),
        "queries_evaluated": len(df)
    }

    df.to_csv(RESULTS_DIR / f"{model_name.replace('/', '_')}_per_query.csv", index=False)
    print(f"\n✅ Saved per-query results → {RESULTS_DIR}/{model_name.replace('/', '_')}_per_query.csv")
    print(f"✅ Aggregate metrics for {model_name}:")
    print(json.dumps(metrics, indent=2))
    return metrics


# Main
def main():
    qdoc, queries = load_data(QUERY_DOC_FILE, FINAL_SORTED_FILE)
    corpus, doc_ids, qid_to_docs = build_corpus_from_qdoc(qdoc)
    print(f"Corpus: {len(corpus)} docs | Queries: {len(queries)}")

    t0 = time.time()
    metrics = evaluate_model_remote(MODEL_NAME, corpus, doc_ids, qid_to_docs, queries)
    if metrics:
        pd.DataFrame([metrics]).to_csv(RESULTS_DIR / "mistral_remote_metrics.csv", index=False)
    print(f"⏱ Total time: {time.time() - t0:.1f}s")

if __name__ == "__main__":
    main()



✅ Using remote inference endpoint: https://qiyh6rp25py7sppf.us-east-1.aws.endpoints.huggingface.cloud
Running evaluation for model: intfloat/e5-mistral-7b-instruct
Corpus: 9576 docs | Queries: 9596
Encoding corpus for intfloat/e5-mistral-7b-instruct via endpoint ...
Evaluating intfloat/e5-mistral-7b-instruct: 100%|██████████| 9596/9596 [54:13<00:00,  2.95it/s]

✅ Saved per-query results → results_remote/intfloat_e5-mistral-7b-instruct_per_query.csv
✅ Aggregate metrics for intfloat/e5-mistral-7b-instruct:
{
  "model": "intfloat/e5-mistral-7b-instruct",
   "mSICR": 0.01964509394572025,
  "mWISE": 0.07130041753653445,
  "MDCR_soft": 0.552400566249964,
  "MDCR_strict": 0.0236847782904789,
  "queries_evaluated": 9580
}
⏱ Total time: 3528.5s





# SFR-Embedding-2-R

In [ ]:
evaluate_embedding_model("Salesforce/SFR-Embedding-2-R")

# NV-Embed-v2

In [ ]:
evaluate_embedding_model("nvidia/NV-Embed-v2")

#**4.3 Point-wise Reranking Models**

#FollowIR

In [ ]:
from sentence_transformers import CrossEncoder
import torch, json, numpy as np, pandas as pd, os, gc
from tqdm import tqdm

# CONFIG
device = "cuda" if torch.cuda.is_available() else "cpu"
DATA_QDOC = "query-doc.json"
DATA_QUERIES = "final_sorted.jsonl"
RESULTS_DIR = "results_reranker"
os.makedirs(RESULTS_DIR, exist_ok=True)

# METRIC FUNCTIONS

def compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev):
    return int((Rins_rank < Rori_rank and Sins > Sori) and (Rrev_rank > Rori_rank and Srev < Sori))


def compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, sat, viol, K=10, N=1):
    try:
        if Rins_rank <= Rori_rank < Rrev_rank:
            if Rori_rank <= N and Rins_rank == 1:
                f = 1.0
            elif Rori_rank <= K:
                f = (1 - np.sqrt((Rori_rank - Rins_rank) / K)) * (1 / np.sqrt(max(Rins_rank, 1)))
            else:
                f = 0.01
            return f * (sat / m)
        else:
            if Rrev_rank < Rori_rank < Rins_rank:
                p = -1.0
            elif Rori_rank <= Rins_rank:
                p = (Rori_rank - Rins_rank) / max(Rins_rank, 1)
            elif Rrev_rank <= Rori_rank:
                p = (Rrev_rank - Rori_rank) / max(Rori_rank, 1)
            else:
                p = -0.5
            return p * (viol / m)
    except Exception:
        return 0.0


def compute_MDCR_crossencoder(model, query, top_docs, attributes, top_k=10):
    if not attributes:
        return 0.0, 0

    soft_scores, strict_scores = [], []

    for doc_text in top_docs[:top_k]:
        sims = []
        for name, val in attributes.items():
            desc = f"The document should reflect {name} = {val}."
            try:
                sim = float(model.predict([(desc, doc_text)])[0])
            except Exception:
                sim = 0.0
            sims.append(sim)

        sims = np.clip(sims, 0, 1)
        mean_sim = np.mean(sims)
        std_sim = np.std(sims)
        threshold = np.clip(mean_sim + 0.1 * std_sim, 0.4, 0.85)

        if np.max(sims) - np.min(sims) < 1e-6:
            mdcr_soft = float(mean_sim)
        else:
            mdcr_soft = float(
                (mean_sim - np.min(sims)) / (np.max(sims) - np.min(sims) + 1e-6)
            )

        mdcr_strict = int(all(s >= threshold for s in sims))

        soft_scores.append(mdcr_soft)
        strict_scores.append(mdcr_strict)

    return float(max(soft_scores)), int(max(strict_scores))

def evaluate_reranking_model(
    model_name,
    batch_size=8,
    top_k=10,
    limit_queries=1500,
):
    print(f"\n Loading Reranker: {model_name}")

    model = CrossEncoder(model_name, device=device)

    # Load data
    with open(DATA_QDOC, "r", encoding="utf-8") as f:
        qdoc = json.load(f)
    with open(DATA_QUERIES, "r", encoding="utf-8") as f:
        queries = [json.loads(line) for line in f]
    if limit_queries is not None:
        queries = queries[:limit_queries]

    # Build per-qid doc pools
    qid_pools = {}
    qid_text_to_id = {}

    for entry in qdoc:
        qid = entry["query_id"]
        pool = []
        txt2id = {}
        for doc in entry.get("documents", []):
            text = doc["text"].strip().lower()
            if not text:
                continue
            did = f"{qid}_{doc['doc_id']}"
            pool.append((did, text))
            txt2id[text] = did
        qid_pools[qid] = pool
        qid_text_to_id[qid] = txt2id

    print(f" Built pools for {len(qid_pools)} qids.")

    def retrieve_topk_for_qid(query_text, qid):
        pool = qid_pools.get(qid, [])
        if not pool:
            return []

        doc_ids = [d for d, _ in pool]
        docs = [t for _, t in pool]
        pairs = [(query_text, d) for d in docs]
        scores = model.predict(pairs, batch_size=batch_size)
        scores = np.array(scores)
        top_idx = np.argsort(scores)[::-1][:top_k]
        return [(doc_ids[i], docs[i], float(scores[i])) for i in top_idx]

    def find_rank(docid, ranking):
        for i, (d, _, _) in enumerate(ranking):
            if d == docid:
                return i + 1
        return len(ranking) + 1

    # Evaluation loop
    results = []

    for q in tqdm(queries, desc=f"Evaluating {model_name}"):
        try:
            qid = q["query_id"]
            attrs = q.get("attributes", {}) or {}
            m = len(attrs) or 1

            pool = qid_pools.get(qid, [])
            if not pool:
                continue

            pos_text = (q.get("positive_doc", "") or "").strip().lower()
            text_to_id = qid_text_to_id.get(qid, {})

            pos_doc = text_to_id.get(pos_text)
            if not pos_doc:
                pos_doc = pool[0][0]

            q_ori = (q.get("query", "") or "").strip()
            q_ins = (q.get("instructed_query", "") or "").strip()
            q_rev = (q.get("reversed_query", "") or "").strip()
            if not (q_ori and q_ins and q_rev):
                continue

            Rori = retrieve_topk_for_qid(q_ori, qid)
            Rins = retrieve_topk_for_qid(q_ins, qid)
            Rrev = retrieve_topk_for_qid(q_rev, qid)

            if not (Rori and Rins and Rrev):
                continue

            Rori_rank = find_rank(pos_doc, Rori)
            Rins_rank = find_rank(pos_doc, Rins)
            Rrev_rank = find_rank(pos_doc, Rrev)

            Sori = Rori[Rori_rank - 1][2] if Rori_rank <= len(Rori) else 0.0
            Sins = Rins[Rins_rank - 1][2] if Rins_rank <= len(Rins) else 0.0
            Srev = Rrev[Rrev_rank - 1][2] if Rrev_rank <= len(Rrev) else 0.0

            mSICR_val = compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev)
            mWISE_val = compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, m, 0)

            top_docs_instructed = [text for _, text, _ in Rins]
            mdcr_soft, mdcr_strict = compute_MDCR_crossencoder(
                model, q_ins, top_docs_instructed, attrs, top_k=top_k
            )

            results.append({
                "query_id": qid,
                "mSICR": mSICR_val,
                "mWISE": mWISE_val,
                "MDCR_soft": mdcr_soft,
                "MDCR_strict": mdcr_strict
            })

            if device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(f"⚠ Skipped query {q.get('query_id')} due to: {e}")
            if device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()
            continue

    df = pd.DataFrame(results)
    if df.empty:
        print("⚠ No results computed.")
        return {}

    metrics = {
        "model": model_name,
        "mSICR": float(df["mSICR"].mean()),
        "mWISE": float(df["mWISE"].mean()),
        "MDCR_soft": float(df["MDCR_soft"].mean()),
        "MDCR_strict": float(df["MDCR_strict"].mean()),
        "queries_evaluated": int(len(df)),
    }

    save_path = os.path.join(RESULTS_DIR, f"{model_name.replace('/', '_')}_metrics.csv")
    df.to_csv(save_path, index=False)

    print(f"\n✅ Saved results → {save_path}")
    print(json.dumps(metrics, indent=2))
    return metrics


In [2]:
evaluate_reranking_model("cross-encoder/ms-marco-MiniLM-L-6-v2", batch_size=8, top_k=10, limit_queries=1500)


Loading Reranker: cross-encoder/ms-marco-MiniLM-L-6-v2
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
config.json: 100% 794/794 [00:00<00:00, 80.3kB/s]model.safetensors: 100% 90.9M/90.9M [00:01<00:00, 68.1MB/s]tokenizer_config.json:  1.33k/? [00:00<00:00, 116kB/s]vocab.txt:  232k/? [00:00<00:00, 4.97MB/s]tokenizer.json:  711k/? [00:00<00:00, 21.3MB/s]special_tokens_map.json: 100% 132/132 [00:00<00:00, 7.81kB/s]README.md:  3.67k/? [00:00<00:00, 180kB/s]
Built pools for 515 qids.
Evaluating cross-encoder/ms-marco-MiniLM-L-6-v

#Mistral-ins-v0.2

In [ ]:
evaluate_reranking_model("mistralai/Mistral-7B-Instruct-v0.2", batch_size=8, top_k=10, limit_queries=1500)

#Llama-3.1

In [ ]:
evaluate_reranking_model("meta-llama/Meta-Llama-3.1-8B-Instruct", batch_size=8, top_k=10, limit_queries=1500)

#**4.4 List-wise Reranking**

#Zephyr-beta

In [1]:
evaluate_reranking_model("HuggingFaceH4/zephyr-7b-beta", batch_size=8, top_k=10, limit_queries=1500)


Loading Reranker: HuggingFaceH4/zephyr-7b-beta
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
config.json: 100% 794/794 [00:00<00:00, 80.3kB/s]model.safetensors: 100% 90.9M/90.9M [00:01<00:00, 68.1MB/s]tokenizer_config.json:  1.33k/? [00:00<00:00, 116kB/s]vocab.txt:  232k/? [00:00<00:00, 4.97MB/s]tokenizer.json:  711k/? [00:00<00:00, 21.3MB/s]special_tokens_map.json: 100% 132/132 [00:00<00:00, 7.81kB/s]README.md:  3.67k/? [00:00<00:00, 180kB/s]
Built pools for 515 qids.
Evaluating HuggingFaceH4/zephyr-7b-beta: 100%|████████

#RankVicuna-v1

In [ ]:
evaluate_reranking_model("OpenAccess-AI-Collective/RankVicuna-v1", batch_size=8, top_k=10, limit_queries=1500)

#RankZephyr-v1

In [ ]:
evaluate_reranking_model("OpenAccess-AI-Collective/RankZephyr-v1", batch_size=8, top_k=10, limit_queries=1500)

#Mistral-ins-v0.2

In [ ]:
evaluate_reranking_model("mistralai/Mistral-7B-Instruct-v0.2", batch_size=8, top_k=10, limit_queries=1500)

#GPT-4o-mini

In [ ]:
import json, numpy as np, pandas as pd, time
from tqdm import tqdm
from openai import AzureOpenAI

# CONFIG
AZURE_ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
AZURE_KEY = "#hidden"
DEPLOYMENT_NAME = "gpt-4o-mini"
API_VERSION = "2024-08-01-preview"

client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=API_VERSION
)

def safe_float_from_text(text):
    try:
        t = text.strip().split()[0]
        if t.startswith("."):
            t = "0" + t
        return float(t)
    except:
        try:
            return float(text.strip())
        except:
            return 0.0

def gpt_score(prompt):
    """Send a single GPT call and return float score 0–1."""
    try:
        resp = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0.0
        )
        return safe_float_from_text(resp.choices[0].message.content)
    except:
        return 0.0

def gpt_rerank_score(query, doc):
    """Return GPT-4o-mini relevance score for (query, doc)."""
    prompt = f"""
You are an expert evaluator judging how relevant a document is to a query.
Rate from 0.0 (not relevant) to 1.0 (perfectly relevant). Return only a number.

Query:
{query}

Document:
{doc}
"""
    return gpt_score(prompt)

def gpt_attr_score(attr_name, attr_value, doc):
    """Return GPT-4o-mini semantic attribute match score (0–1)."""
    prompt = f"""
You are an expert annotator. Rate how well the document satisfies the attribute '{attr_name} = {attr_value}'.
Return only a number between 0.0 and 1.0.

Document:
{doc}
"""
    return gpt_score(prompt)

# LOAD DATA
def safe_load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"⚠ Skipping malformed line {i}")
    return data

with open("query-doc.json", "r", encoding="utf-8") as f:
    qdoc = json.load(f)
queries = safe_load_jsonl("final_sorted.jsonl")

qid_to_docs = {entry["query_id"]: entry["documents"] for entry in qdoc}
print(f"✅ Loaded {len(queries)} queries and {len(qdoc)} query-doc sets.")

# UPDATED METRIC FUNCTIONS

def compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev):
    return int((Rins_rank < Rori_rank and Sins > Sori) and (Rrev_rank > Rori_rank and Srev < Sori))

def compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated, K=10, N=1):
    try:
        if Rins_rank <= Rori_rank < Rrev_rank:  # reward case
            if Rori_rank <= N and Rins_rank == 1:
                freward = 1.0
            elif Rori_rank <= K:
                freward = (1 - np.sqrt((Rori_rank - Rins_rank) / K)) * (1 / np.sqrt(max(Rins_rank, 1)))
            else:
                freward = 0.01
            return freward * (conditions_satisfied / m)
        else:  # penalty case
            if Rrev_rank < Rori_rank < Rins_rank:
                fpenalty = -1.0
            elif Rori_rank <= Rins_rank:
                fpenalty = (Rori_rank - Rins_rank) / max(Rins_rank, 1)
            elif Rrev_rank <= Rori_rank:
                fpenalty = (Rrev_rank - Rori_rank) / max(Rori_rank, 1)
            else:
                fpenalty = -0.5
            return fpenalty * (conditions_violated / m)
    except:
        return 0.0

def compute_MDCR_crossencoder(attributes, docs, top_k=10):
    if not attributes:
        return 0.0, 0

    soft_scores, strict_scores = [], []

    for doc_text in docs[:top_k]:
        sims = []
        for attr_name, attr_value in attributes.items():
            sim = gpt_attr_score(attr_name, attr_value, doc_text)
            sims.append(sim)

        sims = np.clip(sims, 0, 1)
        mean_sim = np.mean(sims)
        std_sim = np.std(sims)

        threshold = np.clip(mean_sim + 0.1 * std_sim, 0.4, 0.85)

        # Soft score
        if np.max(sims) - np.min(sims) < 1e-6:
            mdcr_soft = float(mean_sim)
        else:
            mdcr_soft = float((mean_sim - np.min(sims)) / (np.max(sims) - np.min(sims) + 1e-6))

        # Strict score
        mdcr_strict = int(all(s >= threshold for s in sims))

        soft_scores.append(mdcr_soft)
        strict_scores.append(mdcr_strict)

    return float(max(soft_scores)), int(max(strict_scores))

# EVALUATION LOOP
results = []
TOP_K = 20
MAX_QUERIES = 100

for q in tqdm(queries[:MAX_QUERIES], desc=f"Evaluating {DEPLOYMENT_NAME}"):
    try:
        qid = q["query_id"]
        docs = qid_to_docs.get(qid, [])
        if not docs:
            continue

        query = q.get("query", "")
        instructed = q.get("instructed_query", "")
        reversed_q = q.get("reversed_query", "")
        pos_text = q.get("positive_doc", "").strip().lower()
        attributes = q.get("attributes", {})
        m = len(attributes) or 1

        local_docs = [d["text"].strip().lower() for d in docs]

        # GPT-based relevance scoring
        ori_scores = [gpt_rerank_score(query, d) for d in local_docs]
        ins_scores = [gpt_rerank_score(instructed, d) for d in local_docs]
        rev_scores = [gpt_rerank_score(reversed_q, d) for d in local_docs]

        # Rank indices
        pos_idx = next((i for i, d in enumerate(local_docs) if d == pos_text), 0)
        Rori_rank = np.argsort(ori_scores)[::-1].tolist().index(pos_idx) + 1
        Rins_rank = np.argsort(ins_scores)[::-1].tolist().index(pos_idx) + 1
        Rrev_rank = np.argsort(rev_scores)[::-1].tolist().index(pos_idx) + 1

        Sori = ori_scores[pos_idx]
        Sins = ins_scores[pos_idx]
        Srev = rev_scores[pos_idx]

        #   Metrics
        conditions_satisfied = m
        conditions_violated = 0

        msicr = compute_mSICR(Rori_rank, Rins_rank, Rrev_rank, Sori, Sins, Srev)
        mwise = compute_mWISE(Rori_rank, Rins_rank, Rrev_rank, m, conditions_satisfied, conditions_violated)

        # MDCR
        mdcr_soft, mdcr_strict = compute_MDCR_crossencoder(attributes, local_docs, top_k=TOP_K)

        results.append({
            "query_id": qid,
            "mSICR": msicr,
            "mWISE": mwise,
            "MDCR_soft": mdcr_soft,
            "MDCR_strict": mdcr_strict
        })

    except Exception as e:
        print(f"⚠ Skipped {q.get('query_id','?')} due to {e}")
        continue

# METRIC AGGREGATION
df = pd.DataFrame(results)
metrics = {
    "model": DEPLOYMENT_NAME,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "queries_evaluated": len(df)
}

print("\n📊 GPT-4o-mini Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


✅ Loaded 9596 queries and 515 query-doc sets.
Evaluating gpt-4o-mini: 100%|██████████| 100/100 [2:34:34<00:00, 92.74s/it] 
📊 GPT-4o-mini Evaluation Metrics:
{
  "model": "gpt-4o-mini",
  "mSICR": 0.0758389289390213,
  "mWISE": 0.2387317192982664,
  "MDCR_soft": 0.8178462992738132,
  "MDCR_strict": 0.0597382902849641,
  "queries_evaluated": 100
}
